## Data loading

In [0]:
# Load Larger LSTM network and generate textimport sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls gdrive/"My Drive"/"EIP 3"/"phase 2"/"A2"

weights-improvement-01-1.5494-bigger.hdf5
weights-improvement-01-1.6597-bigger.hdf5
weights-improvement-01-2.7005-bigger.hdf5
weights-improvement-01-2.8524-bigger.hdf5
weights-improvement-02-1.5372-bigger.hdf5
weights-improvement-02-1.6467-bigger.hdf5
weights-improvement-02-2.5432-bigger.hdf5
weights-improvement-03-1.5267-bigger.hdf5
weights-improvement-03-1.6373-bigger.hdf5
weights-improvement-03-2.4100-bigger.hdf5
weights-improvement-04-1.5219-bigger.hdf5
weights-improvement-04-1.6250-bigger.hdf5
weights-improvement-04-2.2997-bigger.hdf5
weights-improvement-05-1.5091-bigger.hdf5
weights-improvement-05-1.6141-bigger.hdf5
weights-improvement-05-2.2144-bigger.hdf5
weights-improvement-06-1.5070-bigger.hdf5
weights-improvement-06-1.6006-bigger.hdf5
weights-improvement-06-2.1451-bigger.hdf5
weights-improvement-07-1.4891-bigger.hdf5
weights-improvement-07-1.5858-bigger.hdf5
weights-improvement-07-2.0896-bigger.hdf5
weights-improvement-08-1.4846-bigger.hdf5
weights-improvement-08-1.5773-bigg

In [5]:
!wc -l gdrive/"My Drive"/"EIP 3"/"phase 2"/"A2"/wonderland.txt

3330 gdrive/My Drive/EIP 3/phase 2/A2/wonderland.txt


In [0]:
# load ascii text and covert to lowercase
filename = "gdrive/My Drive/EIP 3/phase 2/A2/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))

In [0]:
chars.remove('!')
chars.remove('"')
chars.remove('(')
chars.remove(')')
chars.remove('*')
chars.remove(',')
chars.remove('-')
chars.remove(':')
chars.remove(';')
chars.remove('?')
chars.remove('[')
chars.remove(']')
chars.remove('_')
chars.remove('\'')
chars.remove('\r')

In [9]:
chars

['\n',
 ' ',
 '.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [0]:
raw_text = raw_text.replace("!", "")
raw_text = raw_text.replace("\"", "")
raw_text = raw_text.replace("(", "")
raw_text = raw_text.replace(")", "")
raw_text = raw_text.replace("*", "")
raw_text = raw_text.replace(",", "")
raw_text = raw_text.replace("-", "")
raw_text = raw_text.replace(":", "")
raw_text = raw_text.replace(";", "")
raw_text = raw_text.replace("?", "")
raw_text = raw_text.replace("[", "")
raw_text = raw_text.replace("]", "")
raw_text = raw_text.replace("_", "")
raw_text = raw_text.replace("\r", "")
raw_text = raw_text.replace("'", "")

In [11]:
raw_text[:1000]

'chapter i. down the rabbithole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank and of having nothing to do once or twice she had peeped into the\nbook her sister was reading but it had no pictures or conversations in\nit and what is the use of a book thought alice without pictures or\nconversations\n\nso she was considering in her own mind as well as she could for the\nhot day made her feel very sleepy and stupid whether the pleasure\nof making a daisychain would be worth the trouble of getting up and\npicking the daisies when suddenly a white rabbit with pink eyes ran\nclose by her.\n\nthere was nothing so very remarkable in that nor did alice think it so\nvery much out of the way to hear the rabbit say to itself oh dear\noh dear i shall be late when she thought it over afterwards it\noccurred to her that she ought to have wondered at this but at the time\nit all seemed quite natural but when the rabbit actually took a watch\nout of its waistcoatpocket 

In [0]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [13]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

('Total Characters: ', 137014)
('Total Vocab: ', 29)


In [0]:
splits = raw_text.split('.')

In [15]:
splits[:10]

['chapter i',
 ' down the rabbithole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank and of having nothing to do once or twice she had peeped into the\nbook her sister was reading but it had no pictures or conversations in\nit and what is the use of a book thought alice without pictures or\nconversations\n\nso she was considering in her own mind as well as she could for the\nhot day made her feel very sleepy and stupid whether the pleasure\nof making a daisychain would be worth the trouble of getting up and\npicking the daisies when suddenly a white rabbit with pink eyes ran\nclose by her',
 '\n\nthere was nothing so very remarkable in that nor did alice think it so\nvery much out of the way to hear the rabbit say to itself oh dear\noh dear i shall be late when she thought it over afterwards it\noccurred to her that she ought to have wondered at this but at the time\nit all seemed quite natural but when the rabbit actually took a watch\nout of its waistco

In [0]:
def get_batch_item(string, seq_length):
  dataX = []
  dataY = []
  if len(string)>seq_length :
    for i in range(0, len(string) - seq_length, 1):
      seq_in = string[i:i + seq_length]
      seq_out = string[i + seq_length]
      dataX.append(seq_in)
      dataY.append(seq_out)
  #     dataX.append([char_to_int[char] for char in seq_in])
  #     dataY.append(char_to_int[seq_out])
  else:
    dataX.append(string[:-1])
    dataY.append(string[-1])
  return dataX, dataY

In [17]:
get_batch_item("kiran.", 3)

(['kir', 'ira', 'ran'], ['a', 'n', '.'])

In [18]:
get_batch_item("kiran", 6)

(['kira'], ['n'])

In [19]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 70
dataX = []
dataY = []
for split in splits:
  Xs, Ys = get_batch_item(split, seq_length)
  for x in Xs:
#       dataX.append(x)
    dataX.append([char_to_int[char] for char in x])
  for y in Ys:
#       dataY.append(y)
    dataY.append(char_to_int[y])
n_patterns = len(dataX)
y_patterns = len(dataY)
print "Total Patterns: ", n_patterns , y_patterns 

Total Patterns:  76916 76916


In [0]:
from keras.preprocessing.sequence import pad_sequences
dataX = pad_sequences(dataX)

In [21]:
print([len(x) for x in dataX[:5]])

[70, 70, 70, 70, 70]


In [22]:
[x for x in dataY[:5]]

[11, 1, 4, 27, 1]

## Model

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="gdrive/My Drive/EIP 3/phase 2/A2/"+"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
# fit the model
model.fit(X, y, epochs=1, batch_size=128, callbacks=callbacks_list)

W0723 17:24:35.237792 139889999755136 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 17:24:35.244981 139889999755136 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0723 17:24:35.258846 139889999755136 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0723 17:24:35.278593 139889999755136 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backe

Epoch 1/1
76916/76916 [==============================] - 167s 2ms/step - loss: 2.8524

Epoch 00001: loss improved from inf to 2.85243, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-01-2.8524-bigger.hdf5


In [0]:
model.fit(X, y, epochs=24, batch_size=128, callbacks=callbacks_list)

Epoch 1/24
76916/76916 [==============================] - 167s 2ms/step - loss: 2.7005

Epoch 00001: loss improved from 2.85243 to 2.70050, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-01-2.7005-bigger.hdf5
Epoch 2/24
76916/76916 [==============================] - 165s 2ms/step - loss: 2.5432

Epoch 00002: loss improved from 2.70050 to 2.54322, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-02-2.5432-bigger.hdf5
Epoch 3/24
76916/76916 [==============================] - 166s 2ms/step - loss: 2.4100

Epoch 00003: loss improved from 2.54322 to 2.41000, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-03-2.4100-bigger.hdf5
Epoch 4/24
76916/76916 [==============================] - 163s 2ms/step - loss: 2.2997

Epoch 00004: loss improved from 2.41000 to 2.29973, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-04-2.2997-bigger.hdf5
Epoch 5/24
76916/76916 [==============================] - 162s 2ms/step - lo

In [0]:
# load the network weights
filename = "gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-01-2.8524-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
import sys
import numpy
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
output=""
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
#   print(x.shape)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  output = output+result
  pattern = numpy.append(pattern , index)
  pattern = pattern[1:len(pattern)]
print(str(output))
print "\nDone."

Seed:
" all
think nothing of tumbling down stairs how brave theyll all think m "
e gor ier iead to seadi oe the had been wotle sather anl ereatu and seadi oe the had noted ano anice anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anlce anl

Done.


## Epochs 25-35

In [0]:
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
76916/76916 [==============================] - 164s 2ms/step - loss: 1.6597

Epoch 00001: loss improved from 1.67320 to 1.65972, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-01-1.6597-bigger.hdf5
Epoch 2/10
76916/76916 [==============================] - 164s 2ms/step - loss: 1.6467

Epoch 00002: loss improved from 1.65972 to 1.64672, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-02-1.6467-bigger.hdf5
Epoch 3/10
76916/76916 [==============================] - 163s 2ms/step - loss: 1.6373

Epoch 00003: loss improved from 1.64672 to 1.63730, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-03-1.6373-bigger.hdf5
Epoch 4/10
76916/76916 [==============================] - 164s 2ms/step - loss: 1.6250

Epoch 00004: loss improved from 1.63730 to 1.62503, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-04-1.6250-bigger.hdf5
Epoch 5/10
76916/76916 [==============================] - 164s 2ms/step - lo

In [0]:
import sys
import numpy
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
output=""
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
#   print(x.shape)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  output = output+result
  pattern = numpy.append(pattern , index)
  pattern = pattern[1:len(pattern)]
print(str(output))
print "\nDone."

Seed:
" e taught us said the mock turtle
angrily really you are very dull

you "
 suop anice anlce as seallng tometny temember the hell hoooowed to eone oear the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge hardenert the whe marge h

Done.


## Epochs 35-50

In [0]:
model.fit(X, y, epochs=15, batch_size=128, callbacks=callbacks_list)

Epoch 1/15
76916/76916 [==============================] - 165s 2ms/step - loss: 1.5494

Epoch 00001: loss improved from 1.55584 to 1.54935, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-01-1.5494-bigger.hdf5
Epoch 2/15
76916/76916 [==============================] - 165s 2ms/step - loss: 1.5372

Epoch 00002: loss improved from 1.54935 to 1.53725, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-02-1.5372-bigger.hdf5
Epoch 3/15
76916/76916 [==============================] - 164s 2ms/step - loss: 1.5267

Epoch 00003: loss improved from 1.53725 to 1.52668, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-03-1.5267-bigger.hdf5
Epoch 4/15
76916/76916 [==============================] - 164s 2ms/step - loss: 1.5219

Epoch 00004: loss improved from 1.52668 to 1.52192, saving model to gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-04-1.5219-bigger.hdf5
Epoch 5/15
76916/76916 [==============================] - 164s 2ms/step - lo

In [31]:
# load the network weights
filename = "gdrive/My Drive/EIP 3/phase 2/A2/weights-improvement-14-1.4385-bigger.hdf5"
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=0, batch_size=128, callbacks=callbacks_list)

In [0]:
model.load_weights(filename)

In [29]:
import sys
import numpy
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
output=""
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
#   print(x.shape)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  output = output+result
  pattern = numpy.append(pattern , index)
  pattern = pattern[1:len(pattern)]
print(str(output))
print "\nDone."

Seed:
" f what such an extraordinary ways of
living would be like but it puzzl "
ing ier anice suiet suomeu tealln separking tometie brd seadiises suiet anice sepper anice suiet suomeu tealln separking tomet teparking oureer anice sepper anice suiet suomeu tealln separking tometie brd seadiises suiet anice sepper anice suiet suomeu tealln separking tomet teparking oureer anice sepper anice suiet suomeu tealln separking tometie brd seadiises suiet anice sepper anice suiet suomeu tealln separking tomet teparking oureer anice sepper anice suiet suomeu tealln separking tometie b

Done.


## Epochs 50-60

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="gdrive/My Drive/EIP 3/phase 2/A2/50+"+"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [33]:
model.fit(X, y, epochs=10, batch_size=256, callbacks=callbacks_list)

Epoch 1/10
76916/76916 [==============================] - 94s 1ms/step - loss: 1.4084

Epoch 00001: loss improved from inf to 1.40844, saving model to gdrive/My Drive/EIP 3/phase 2/A2/50+weights-improvement-01-1.4084-bigger.hdf5
Epoch 2/10
76916/76916 [==============================] - 93s 1ms/step - loss: 1.4001

Epoch 00002: loss improved from 1.40844 to 1.40007, saving model to gdrive/My Drive/EIP 3/phase 2/A2/50+weights-improvement-02-1.4001-bigger.hdf5
Epoch 3/10
76916/76916 [==============================] - 93s 1ms/step - loss: 1.3902

Epoch 00003: loss improved from 1.40007 to 1.39023, saving model to gdrive/My Drive/EIP 3/phase 2/A2/50+weights-improvement-03-1.3902-bigger.hdf5
Epoch 4/10
76916/76916 [==============================] - 93s 1ms/step - loss: 1.3828

Epoch 00004: loss improved from 1.39023 to 1.38280, saving model to gdrive/My Drive/EIP 3/phase 2/A2/50+weights-improvement-04-1.3828-bigger.hdf5
Epoch 5/10
76916/76916 [==============================] - 93s 1ms/step -

In [0]:
# define the checkpoint
filepath="gdrive/My Drive/EIP 3/phase 2/A2/60+"+"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Epochs 60-70

In [35]:
model.fit(X, y, epochs=10, batch_size=512, callbacks=callbacks_list)

Epoch 1/10
76916/76916 [==============================] - 58s 759us/step - loss: 1.3170

Epoch 00001: loss improved from inf to 1.31697, saving model to gdrive/My Drive/EIP 3/phase 2/A2/60+weights-improvement-01-1.3170-bigger.hdf5
Epoch 2/10
76916/76916 [==============================] - 58s 760us/step - loss: 1.3019

Epoch 00002: loss improved from 1.31697 to 1.30185, saving model to gdrive/My Drive/EIP 3/phase 2/A2/60+weights-improvement-02-1.3019-bigger.hdf5
Epoch 3/10
76916/76916 [==============================] - 58s 760us/step - loss: 1.2996

Epoch 00003: loss improved from 1.30185 to 1.29960, saving model to gdrive/My Drive/EIP 3/phase 2/A2/60+weights-improvement-03-1.2996-bigger.hdf5
Epoch 4/10
76916/76916 [==============================] - 58s 759us/step - loss: 1.2996

Epoch 00004: loss improved from 1.29960 to 1.29957, saving model to gdrive/My Drive/EIP 3/phase 2/A2/60+weights-improvement-04-1.2996-bigger.hdf5
Epoch 5/10
76916/76916 [==============================] - 58s 75

In [0]:
# define the checkpoint
filepath="gdrive/My Drive/EIP 3/phase 2/A2/70+"+"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Epochs 70-80

In [38]:
model.fit(X, y, epochs=10, batch_size=1024, callbacks=callbacks_list)

Epoch 1/10
76916/76916 [==============================] - 45s 579us/step - loss: 1.2672

Epoch 00001: loss improved from inf to 1.26722, saving model to gdrive/My Drive/EIP 3/phase 2/A2/70+weights-improvement-01-1.2672-bigger.hdf5
Epoch 2/10
76916/76916 [==============================] - 44s 577us/step - loss: 1.2597

Epoch 00002: loss improved from 1.26722 to 1.25968, saving model to gdrive/My Drive/EIP 3/phase 2/A2/70+weights-improvement-02-1.2597-bigger.hdf5
Epoch 3/10
76916/76916 [==============================] - 44s 577us/step - loss: 1.2509

Epoch 00003: loss improved from 1.25968 to 1.25095, saving model to gdrive/My Drive/EIP 3/phase 2/A2/70+weights-improvement-03-1.2509-bigger.hdf5
Epoch 4/10
76916/76916 [==============================] - 44s 577us/step - loss: 1.2486

Epoch 00004: loss improved from 1.25095 to 1.24858, saving model to gdrive/My Drive/EIP 3/phase 2/A2/70+weights-improvement-04-1.2486-bigger.hdf5
Epoch 5/10
76916/76916 [==============================] - 44s 57

## Epochs 80-100

In [39]:
# define the checkpoint
filepath="gdrive/My Drive/EIP 3/phase 2/A2/80+"+"weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=20, batch_size=1024, callbacks=callbacks_list)

Epoch 1/20
76916/76916 [==============================] - 45s 579us/step - loss: 1.2370

Epoch 00001: loss improved from inf to 1.23697, saving model to gdrive/My Drive/EIP 3/phase 2/A2/80+weights-improvement-01-1.2370-bigger.hdf5
Epoch 2/20
76916/76916 [==============================] - 44s 578us/step - loss: 1.2360

Epoch 00002: loss improved from 1.23697 to 1.23595, saving model to gdrive/My Drive/EIP 3/phase 2/A2/80+weights-improvement-02-1.2360-bigger.hdf5
Epoch 3/20
76916/76916 [==============================] - 44s 576us/step - loss: 1.2374

Epoch 00003: loss did not improve from 1.23595
Epoch 4/20
76916/76916 [==============================] - 44s 577us/step - loss: 1.2355

Epoch 00004: loss improved from 1.23595 to 1.23550, saving model to gdrive/My Drive/EIP 3/phase 2/A2/80+weights-improvement-04-1.2355-bigger.hdf5
Epoch 5/20
76916/76916 [==============================] - 44s 577us/step - loss: 1.2357

Epoch 00005: loss did not improve from 1.23550
Epoch 6/20
76916/76916 [===

## Final Result

In [40]:
import sys
import numpy
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
output=""
for i in range(500):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
#   print(x.shape)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  output = output+result
  pattern = numpy.append(pattern , index)
  pattern = pattern[1:len(pattern)]
print(str(output))
print "\nDone."

Seed:
" s

im sure those are not the right words said poor alice and her eyes
"
s whe suesteusily anice hellly whats bhter a little bettualent tane whe catee iardenabte atolet oinutes toatied ier whe coove suiet loow anice hellly whats bhter a little bettualent tane whe catee iardenabte atolet oinutes toatied ier whe coove suiet loow anice hellly whats bhter a little bettualent tane whe catee iardenabte atolet oinutes toatied ier whe coove suiet loow anice hellly whats bhter a little bettualent tane whe catee iardenabte atolet oinutes toatied ier whe coove suiet loow anice 

Done.
